In [1]:
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

import gensim
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

Using TensorFlow backend.
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# x_text = np.load('../improv_loso/data/x_text.npy')

In [4]:
# x_text.shape

In [5]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]

EMBEDDING_DIM = 300
# nb_words = 3438
# MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
file_loc =  '../../../dimensionalSM/data/GoogleNews-vectors-negative300.bin'
print (file_loc)

word_vectors = KeyedVectors.load_word2vec_format(file_loc, binary=True)

g_word_embedding_matrix = {}
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=nb_words:
        continue
    try:
        gembedding_vector = word_vectors[word]
        g_word_embedding_matrix[i] = gembedding_vector
    except KeyError:
        g_word_embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)

del(word_vectors)

# print input size
print('Feature shape: ', x_text.shape)
# print('Label shape: ', vad_train.shape)


Found 3427 unique tokens
../../../dimensionalSM/data/GoogleNews-vectors-negative300.bin
Feature shape:  (7166, 300)


In [6]:
vad = np.load('../improv_loso/data/vad.npy')

In [7]:
# standardization
scaled_feature = False

# set Dropout
do = 0.3

# standardized label
scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad


# Concordance correlation coefficient (CCC)-based loss function
# using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [8]:
# split train/test
split = 5733
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)

In [9]:
# API model, if use RNN, first two rnn layer must return_sequences=True
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model1 = text_model1()

In [10]:
# main function for for LOSO
def main(time):
    model_1 = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(x_text[4586:], batch_size=32)
    np.save('5733npy-W2V/text_W2V-5733-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2580).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5733npy-W2V/text_W2V-5733.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 81s 18ms/step - loss: 2.5420 - v_loss: 0.7219 - a_loss: 0.8928 - d_loss: 0.9222 - v_ccc: 0.2755 - a_ccc: 0.1057 - d_ccc: 0.0767 - val_loss: 2.6103 - val_v_loss: 0.8229 - val_a_loss: 0.8696 - val_d_loss: 0.9160 - val_v_ccc: 0.1767 - val_a_ccc: 0.1297 - val_d_ccc: 0.0832
Epoch 2/50
4586/4586 [==============================] - 85s 18ms/step - loss: 2.0155 - v_loss: 0.5633 - a_loss: 0.7053 - d_loss: 0.7464 - v_ccc: 0.4366 - a_ccc: 0.2950 - d_ccc: 0.2529 - val_loss: 2.6365 - val_v_loss: 0.8268 - val_a_loss: 0.8808 - val_d_loss: 0.9274 - val_v_ccc: 0.1724 - val_a_ccc: 0.1186 - val_d_ccc: 0.0725
Epoch 3/50
4586/4586 [==============================] - 88s 19ms/step - loss: 1.7717 - v_loss: 0.4886 - a_loss: 0.6232 - d_loss: 0.6585 - v_ccc: 0.5104 - a_ccc: 0.3764 - d_ccc: 0.3415 - val_loss: 2.7008 - val_v_loss: 0

4586/4586 [==============================] - 82s 18ms/step - loss: 2.5243 - v_loss: 0.7396 - a_loss: 0.8705 - d_loss: 0.9139 - v_ccc: 0.2632 - a_ccc: 0.1290 - d_ccc: 0.0835 - val_loss: 2.5628 - val_v_loss: 0.8135 - val_a_loss: 0.8700 - val_d_loss: 0.8771 - val_v_ccc: 0.1861 - val_a_ccc: 0.1291 - val_d_ccc: 0.1219
Epoch 2/50
4586/4586 [==============================] - 81s 18ms/step - loss: 2.0162 - v_loss: 0.5712 - a_loss: 0.6894 - d_loss: 0.7505 - v_ccc: 0.4277 - a_ccc: 0.3086 - d_ccc: 0.2475 - val_loss: 2.7256 - val_v_loss: 0.8416 - val_a_loss: 0.9148 - val_d_loss: 0.9681 - val_v_ccc: 0.1580 - val_a_ccc: 0.0847 - val_d_ccc: 0.0317
Epoch 3/50
4586/4586 [==============================] - 82s 18ms/step - loss: 1.7616 - v_loss: 0.4962 - a_loss: 0.6046 - d_loss: 0.6582 - v_ccc: 0.5033 - a_ccc: 0.3941 - d_ccc: 0.3410 - val_loss: 2.7305 - val_v_loss: 0.8530 - val_a_loss: 0.9104 - val_d_loss: 0.9646 - val_v_ccc: 0.1453 - val_a_ccc: 0.0891 - val_d_ccc: 0.0352
Epoch 4/50
4586/4586 [===========

4586/4586 [==============================] - 88s 19ms/step - loss: 1.7704 - v_loss: 0.4981 - a_loss: 0.6100 - d_loss: 0.6636 - v_ccc: 0.5033 - a_ccc: 0.3904 - d_ccc: 0.3359 - val_loss: 2.7160 - val_v_loss: 0.8709 - val_a_loss: 0.9142 - val_d_loss: 0.9276 - val_v_ccc: 0.1274 - val_a_ccc: 0.0847 - val_d_ccc: 0.0719
Epoch 4/50
4586/4586 [==============================] - 88s 19ms/step - loss: 1.5803 - v_loss: 0.4426 - a_loss: 0.5564 - d_loss: 0.5843 - v_ccc: 0.5593 - a_ccc: 0.4453 - d_ccc: 0.4151 - val_loss: 2.7890 - val_v_loss: 0.8153 - val_a_loss: 0.9547 - val_d_loss: 1.0172 - val_v_ccc: 0.1836 - val_a_ccc: 0.0443 - val_d_ccc: -0.0168
Epoch 5/50
4586/4586 [==============================] - 89s 19ms/step - loss: 1.4507 - v_loss: 0.3974 - a_loss: 0.5132 - d_loss: 0.5387 - v_ccc: 0.6021 - a_ccc: 0.4866 - d_ccc: 0.4606 - val_loss: 2.8295 - val_v_loss: 0.8407 - val_a_loss: 0.9703 - val_d_loss: 1.0173 - val_v_ccc: 0.1582 - val_a_ccc: 0.0293 - val_d_ccc: -0.0170
Epoch 6/50
4586/4586 [=========

4586/4586 [==============================] - 88s 19ms/step - loss: 1.5882 - v_loss: 0.4438 - a_loss: 0.5599 - d_loss: 0.5862 - v_ccc: 0.5564 - a_ccc: 0.4424 - d_ccc: 0.4130 - val_loss: 2.7491 - val_v_loss: 0.8508 - val_a_loss: 0.9300 - val_d_loss: 0.9662 - val_v_ccc: 0.1473 - val_a_ccc: 0.0699 - val_d_ccc: 0.0337
Epoch 5/50
4586/4586 [==============================] - 88s 19ms/step - loss: 1.4373 - v_loss: 0.3992 - a_loss: 0.5043 - d_loss: 0.5348 - v_ccc: 0.6017 - a_ccc: 0.4946 - d_ccc: 0.4664 - val_loss: 2.7219 - val_v_loss: 0.8281 - val_a_loss: 0.9328 - val_d_loss: 0.9590 - val_v_ccc: 0.1705 - val_a_ccc: 0.0670 - val_d_ccc: 0.0405
Epoch 6/50
4586/4586 [==============================] - 88s 19ms/step - loss: 1.3268 - v_loss: 0.3710 - a_loss: 0.4731 - d_loss: 0.4867 - v_ccc: 0.6310 - a_ccc: 0.5286 - d_ccc: 0.5136 - val_loss: 2.7367 - val_v_loss: 0.8562 - val_a_loss: 0.9187 - val_d_loss: 0.9594 - val_v_ccc: 0.1423 - val_a_ccc: 0.0807 - val_d_ccc: 0.0403
Epoch 7/50
4586/4586 [===========

4586/4586 [==============================] - 86s 19ms/step - loss: 1.5901 - v_loss: 0.4435 - a_loss: 0.5617 - d_loss: 0.5895 - v_ccc: 0.5575 - a_ccc: 0.4407 - d_ccc: 0.4117 - val_loss: 2.7576 - val_v_loss: 0.8489 - val_a_loss: 0.9334 - val_d_loss: 0.9734 - val_v_ccc: 0.1498 - val_a_ccc: 0.0665 - val_d_ccc: 0.0262
Epoch 5/50
4586/4586 [==============================] - 87s 19ms/step - loss: 1.4488 - v_loss: 0.4061 - a_loss: 0.5055 - d_loss: 0.5350 - v_ccc: 0.5925 - a_ccc: 0.4941 - d_ccc: 0.4646 - val_loss: 2.6455 - val_v_loss: 0.8201 - val_a_loss: 0.8868 - val_d_loss: 0.9364 - val_v_ccc: 0.1791 - val_a_ccc: 0.1123 - val_d_ccc: 0.0631
Epoch 6/50
4586/4586 [==============================] - 86s 19ms/step - loss: 1.3029 - v_loss: 0.3660 - a_loss: 0.4645 - d_loss: 0.4759 - v_ccc: 0.6347 - a_ccc: 0.5364 - d_ccc: 0.5260 - val_loss: 2.6657 - val_v_loss: 0.8379 - val_a_loss: 0.9138 - val_d_loss: 0.9113 - val_v_ccc: 0.1612 - val_a_ccc: 0.0854 - val_d_ccc: 0.0876
Epoch 7/50
4586/4586 [===========

4586/4586 [==============================] - 93s 20ms/step - loss: 2.5107 - v_loss: 0.7424 - a_loss: 0.8663 - d_loss: 0.9044 - v_ccc: 0.2577 - a_ccc: 0.1352 - d_ccc: 0.0964 - val_loss: 2.7444 - val_v_loss: 0.8284 - val_a_loss: 0.9446 - val_d_loss: 0.9713 - val_v_ccc: 0.1716 - val_a_ccc: 0.0552 - val_d_ccc: 0.0289
Epoch 2/50
4586/4586 [==============================] - 88s 19ms/step - loss: 2.0242 - v_loss: 0.5782 - a_loss: 0.6975 - d_loss: 0.7535 - v_ccc: 0.4233 - a_ccc: 0.3048 - d_ccc: 0.2477 - val_loss: 2.6145 - val_v_loss: 0.8281 - val_a_loss: 0.8762 - val_d_loss: 0.9089 - val_v_ccc: 0.1708 - val_a_ccc: 0.1239 - val_d_ccc: 0.0908
Epoch 3/50
4586/4586 [==============================] - 89s 19ms/step - loss: 1.7656 - v_loss: 0.4995 - a_loss: 0.6139 - d_loss: 0.6551 - v_ccc: 0.5015 - a_ccc: 0.3872 - d_ccc: 0.3457 - val_loss: 2.6833 - val_v_loss: 0.8363 - val_a_loss: 0.9115 - val_d_loss: 0.9323 - val_v_ccc: 0.1619 - val_a_ccc: 0.0877 - val_d_ccc: 0.0671
Epoch 4/50
4586/4586 [===========

4586/4586 [==============================] - 87s 19ms/step - loss: 2.0465 - v_loss: 0.5656 - a_loss: 0.7164 - d_loss: 0.7619 - v_ccc: 0.4332 - a_ccc: 0.2828 - d_ccc: 0.2374 - val_loss: 2.6681 - val_v_loss: 0.8838 - val_a_loss: 0.8992 - val_d_loss: 0.8830 - val_v_ccc: 0.1150 - val_a_ccc: 0.1003 - val_d_ccc: 0.1167
Epoch 3/50
4586/4586 [==============================] - 88s 19ms/step - loss: 1.7954 - v_loss: 0.5008 - a_loss: 0.6246 - d_loss: 0.6674 - v_ccc: 0.4984 - a_ccc: 0.3745 - d_ccc: 0.3317 - val_loss: 2.7580 - val_v_loss: 0.8641 - val_a_loss: 0.9309 - val_d_loss: 0.9613 - val_v_ccc: 0.1345 - val_a_ccc: 0.0688 - val_d_ccc: 0.0387
Epoch 4/50
4586/4586 [==============================] - 87s 19ms/step - loss: 1.5757 - v_loss: 0.4365 - a_loss: 0.5530 - d_loss: 0.5885 - v_ccc: 0.5635 - a_ccc: 0.4478 - d_ccc: 0.4129 - val_loss: 2.7607 - val_v_loss: 0.8681 - val_a_loss: 0.9346 - val_d_loss: 0.9554 - val_v_ccc: 0.1302 - val_a_ccc: 0.0646 - val_d_ccc: 0.0444
Epoch 5/50
4586/4586 [===========

4586/4586 [==============================] - 88s 19ms/step - loss: 1.7731 - v_loss: 0.4988 - a_loss: 0.6110 - d_loss: 0.6615 - v_ccc: 0.5007 - a_ccc: 0.3873 - d_ccc: 0.3389 - val_loss: 2.6645 - val_v_loss: 0.8633 - val_a_loss: 0.8978 - val_d_loss: 0.9022 - val_v_ccc: 0.1361 - val_a_ccc: 0.1024 - val_d_ccc: 0.0969
Epoch 4/50
4586/4586 [==============================] - 88s 19ms/step - loss: 1.5835 - v_loss: 0.4463 - a_loss: 0.5544 - d_loss: 0.5897 - v_ccc: 0.5544 - a_ccc: 0.4489 - d_ccc: 0.4132 - val_loss: 2.6492 - val_v_loss: 0.8313 - val_a_loss: 0.9052 - val_d_loss: 0.9104 - val_v_ccc: 0.1676 - val_a_ccc: 0.0946 - val_d_ccc: 0.0886
Epoch 5/50
4586/4586 [==============================] - 88s 19ms/step - loss: 1.4341 - v_loss: 0.4008 - a_loss: 0.5002 - d_loss: 0.5329 - v_ccc: 0.5992 - a_ccc: 0.4995 - d_ccc: 0.4673 - val_loss: 2.7145 - val_v_loss: 0.8587 - val_a_loss: 0.8891 - val_d_loss: 0.9642 - val_v_ccc: 0.1397 - val_a_ccc: 0.1102 - val_d_ccc: 0.0356
Epoch 6/50
4586/4586 [===========

1433/1433 [==============================] - 8s 6ms/step
[0.2501172721385956, 0.18101932108402252, 0.14293164014816284]
15
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 94s 21ms/step - loss: 2.4969 - v_loss: 0.7276 - a_loss: 0.8750 - d_loss: 0.8960 - v_ccc: 0.2730 - a_ccc: 0.1258 - d_ccc: 0.1044 - val_loss: 2.6220 - val_v_loss: 0.7962 - val_a_loss: 0.9033 - val_d_loss: 0.9203 - val_v_ccc: 0.2034 - val_a_ccc: 0.0955 - val_d_ccc: 0.0791
Epoch 2/50
4586/4586 [==============================] - 89s 19ms/step - loss: 2.0162 - v_loss: 0.5809 - a_loss: 0.6939 - d_loss: 0.7430 - v_ccc: 0.4203 - a_ccc: 0.3065 - d_ccc: 0.2570 - val_loss: 2.6826 - val_v_loss: 0.8382 - val_a_loss: 0.9170 - val_d_loss: 0.9251 - val_v_ccc: 0.1609 - val_a_ccc: 0.0822 - val_d_ccc: 0.0743
Epoch 3/50
4586/4586 [==============================] - 88s 19ms/step - loss: 1.7502 - v_loss: 0.4908 - a_loss: 0.6105 - d_loss: 0.6478 - v_ccc: 0.5093 - a_ccc: 0.3889 - d_ccc: 

4586/4586 [==============================] - 87s 19ms/step - loss: 2.0020 - v_loss: 0.5664 - a_loss: 0.6898 - d_loss: 0.7462 - v_ccc: 0.4346 - a_ccc: 0.3093 - d_ccc: 0.2541 - val_loss: 2.6066 - val_v_loss: 0.8413 - val_a_loss: 0.8664 - val_d_loss: 0.8968 - val_v_ccc: 0.1576 - val_a_ccc: 0.1331 - val_d_ccc: 0.1027
Epoch 3/50
4586/4586 [==============================] - 87s 19ms/step - loss: 1.7424 - v_loss: 0.4866 - a_loss: 0.6062 - d_loss: 0.6497 - v_ccc: 0.5140 - a_ccc: 0.3942 - d_ccc: 0.3494 - val_loss: 2.6886 - val_v_loss: 0.8295 - val_a_loss: 0.9062 - val_d_loss: 0.9508 - val_v_ccc: 0.1690 - val_a_ccc: 0.0936 - val_d_ccc: 0.0487
Epoch 4/50
4586/4586 [==============================] - 87s 19ms/step - loss: 1.5884 - v_loss: 0.4341 - a_loss: 0.5586 - d_loss: 0.5951 - v_ccc: 0.5671 - a_ccc: 0.4409 - d_ccc: 0.4036 - val_loss: 2.7360 - val_v_loss: 0.9083 - val_a_loss: 0.8806 - val_d_loss: 0.9442 - val_v_ccc: 0.0898 - val_a_ccc: 0.1190 - val_d_ccc: 0.0552
Epoch 5/50
4586/4586 [===========

In [11]:
# hist1 = model1.fit(x_text[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_text[split:], vad[split:].T.tolist())
# print(eval_metrik1[-3:])

# # # make prediction
# # predict = model1.predict(x_text[4596:], batch_size=32)
# # np.save('result_mspin_loso/result_ter_glove', 
# #          np.array(predict).reshape(3, 2570).T)